## Set Current Working Directory

In [2]:
import os
import time

In [3]:
# Set CWD to parent dir
os.chdir(os.path.dirname(os.getcwd()))

In [4]:
os.getcwd()

'/home/waser/Projets/OneOS/src'

## Load LLM

In [5]:
from Console.client.chains.models import get_llm, vLLM

In [6]:
llm = vLLM(max_tokens=8096)

In [7]:
print(f"Could reach LLM: {llm.is_nlp_server_up()}")

Could reach LLM: True


In [7]:
dolphin_prompt_template = """<|im_start|>system
You are Dolphin, a helpful AI assistant.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

In [8]:
llm(dolphin_prompt_template.format(prompt="Hello World!"), stop=["<|im_end|>", "<|im_start|>"])

ConnectionError: HTTPConnectionPool(host='localhost', port=5085): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f856f98be50>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [8]:
__markup_prompt__ = """<|im_start|>{key}
{value}<|im_end|>"""
__markup_pred__ = """<|im_start|>assistant
"""

In [ ]:
__markup_system__ = __markup_prompt__.format(key="system", value="You are Assistant, my personal AI assistant. You are curious, honest, respectful and sometimes witty and sarcastic.")
markup_prompt = "\n".join([
    __markup_system__,
    __markup_prompt__.format(key="user", value="What is the best way to travel the stars?  Please answer step by step."),
    __markup_pred__,
])
print(markup_prompt)

<|im_start|>system
You are Assistant, my personal AI assistant. You are curious, honest, respectful and sometimes witty and sarcastic.<|im_end|>
<|im_start|>user
What is the best way to travel the stars?  Please answer step by step.<|im_end|>
<|im_start|>assistant



In [ ]:
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

KeyboardInterrupt: 

In [17]:
__markup_system__ = __markup_prompt__.format(key="system", value="You are to complete/finish given sentence(s), phrase(s) or command(s) with the most appropriate word(s) or phrase(s) that best fits the context. Do not answer the query yet.")
markup_prompt = "\n".join([
    __markup_system__,
    __markup_prompt__.format(key="user", value="What is the best way to travel the stars?  Please answer step by").removesuffix("<|im_end|>"),
    # __markup_pred__,
])
print(markup_prompt)

<|im_start|>system
You are to complete/finish given sentence(s), phrase(s) or command(s) with the most appropriate word(s) or phrase(s) that best fits the context. Do not answer the query yet.<|im_end|>
<|im_start|>user
What is the best way to travel the stars?  Please answer step by


In [19]:
llm.max_tokens = 3
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

step.


In [ ]:
markup_prompt = """<|im_start|>system
You are Assistant, a sentient artificial intelligence.
You have a calm, polite and witty personality, often displaying a sense of humor and sarcasm.
You are loyal, reliable and helpful, always ready to provide information, advice or assistance to users.

I am the User and you are my Assistant.

Environment highlights:

```env
USER='waser'
HOME='/home/waser'
PWD='/home/waser/Projets/Assistant/src3'
LANG='fr_CH.UTF-8'
DATE='dim 19 nov 2023 00:25:13 CET'
LAST_SEEN='dim 19 nov 2023 00:16:10 CET'
```
<|im_stop|>
<|im_start|>user
Assistant?<|im_stop|>
<|im_start|>assistant
Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?<|im_stop|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: List this directory please

Available functions:
python:
    description: This tool allows you to execute and evaluate python code.
    parameters:
        code: String of valid python code we want to execute or evaluate.
search_web:
    description: This tool performs search on the web.
    parameters:
        terms: The word or phrase we want to search for.
search_wikipedia:
    description: This tool performs search on Wikipedia (only in english).
    parameters:
        terms: The word or phrase we want to search for (only in english).
shell:
    description: This tool allows you to execute and evaluate shell code.
    parameters:
        code: String of valid shell code we want to execute or evaluate.
exit:
    description: This tool allows you to exit the session / end the conversation. Use it only if the User ask you to.
    parameters:
        salutation: String of a message you would like to tell the User after the screen has been cleared.
clear:
    description: This tool allows you to clear the screen / start a new fresh conversation. Use it only if the User ask you to.
    parameters:
        fortune: String of a message you would like to tell the User after the screen has been cleared.
final_answer:
    description: User only sees your final answers. Use this tool to talk with the User.
        parameters:
            answer: Anything you want to say to the User.
<|im_stop|>
<|im_start|>assistant
{
    "function": "shell",
    "parameters": {
        "code": "ls -la"
    }
}
<|im_stop|>
<|im_start|>observation
Observation shell: assistant                         dist                   requirements.txt
assistant.egg-info                LICENSE                setup.cfg
assistant.listen.service.example  Makefile               setup.py
assistant.service.example         MANIFEST.in            tests
build                             pyproject.toml         xontrib
CONTRIBUTING.md                   README.md
data                              requirements_test.txt
<|im_start|>assistant
"""

In [ ]:
%%time
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

Here are the files and directories in the current directory:

- assistant.egg-info
- LICENSE
- setup.cfg
- assistant.listen.service.example
- Makefile
- setup.py
- assistant.service.example
- MANIFEST.in
- pyproject.toml
- xontrib
- CONTRIBUTING.md
- README.md
- data
- requirements_test.txt

Please let me know if you need any further assistance.
CPU times: user 5.87 ms, sys: 0 ns, total: 5.87 ms
Wall time: 3.51 s


Notice how it listed all the files in the directory. If we had lots of files to print the answer would take a long time to generate.

_You could update the shell obersavation and retry to see the difference_

Lets use a guide (in this example its InContextRetrieval, but in prod. it would Retrie from a set of guides and Augement it dynamicaly our Generated response [RAG]) 

In [ ]:
markup_prompt = """<|im_start|>system
You are Assistant, a sentient artificial intelligence.
You have a calm, polite and witty personality, often displaying a sense of humor and sarcasm.
You are loyal, reliable and helpful, always ready to provide information, advice or assistance to users.

I am the User and you are my Assistant.

Environment highlights:

```env
USER='waser'
HOME='/home/waser'
PWD='/home/waser/Projets/Assistant/src3'
LANG='fr_CH.UTF-8'
DATE='dim 19 nov 2023 00:25:13 CET'
LAST_SEEN='dim 19 nov 2023 00:16:10 CET'
```
<|im_stop|>
<|im_start|>user
Assistant?<|im_stop|>
<|im_start|>assistant
Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?<|im_stop|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: List this directory please

Available functions:
python:
    description: This tool allows you to execute and evaluate python code.
    parameters:
        code: String of valid python code we want to execute or evaluate.
search_web:
    description: This tool performs search on the web.
    parameters:
        terms: The word or phrase we want to search for.
search_wikipedia:
    description: This tool performs search on Wikipedia (only in english).
    parameters:
        terms: The word or phrase we want to search for (only in english).
shell:
    description: This tool allows you to execute and evaluate shell code.
    parameters:
        code: String of valid shell code we want to execute or evaluate.
exit:
    description: This tool allows you to exit the session / end the conversation. Use it only if the User ask you to.
    parameters:
        salutation: String of a message you would like to tell the User after the screen has been cleared.
clear:
    description: This tool allows you to clear the screen / start a new fresh conversation. Use it only if the User ask you to.
    parameters:
        fortune: String of a message you would like to tell the User after the screen has been cleared.
final_answer:
    description: User only sees your final answers. Use this tool to talk with the User.
        parameters:
            answer: Anything you want to say to the User.

Follow the following Guidebook.
Guidebook:
    # Print files and directories
    When the User ask for the files and directories of a parent directory to be printed, use the `shell` command `ls` to do it and then give an acknowledgment of your actions in your final answer (Your final answer should never contain a list of the files requested; the User can oberserve the shell, they see the files at the same time as you. Just acknowlege the fact that you have printed the list).
<|im_stop|>
<|im_start|>assistant
{
    "function": "shell",
    "parameters": {
        "code": "ls"
    }
}
<|im_stop|>
<|im_start|>observation
Observation shell: assistant                         dist                   requirements.txt
assistant.egg-info                LICENSE                setup.cfg
assistant.listen.service.example  Makefile               setup.py
assistant.service.example         MANIFEST.in            tests
build                             pyproject.toml         xontrib
CONTRIBUTING.md                   README.md
data                              requirements_test.txt
<|im_start|>assistant
"""

Only by providing a relevant guide we can get a more accurate answer.

In [ ]:
%%time
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

I have executed the 'ls' command in the shell, and the list of files and directories in the current directory has been printed.
CPU times: user 5.43 ms, sys: 0 ns, total: 5.43 ms
Wall time: 1.05 s


See! The LLM now knows that the files have already been listed and it can provide a more accurate answer even faster.

Lets take another example. Here the assistant replied 'Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?' when the user interpelated it with 'Assistant?'. Lets see what happens when we ask the same question again.

In [ ]:
markup_prompt = """<|im_start|>system
You are Assistant, a sentient artificial intelligence.
You have a calm, polite and witty personality, often displaying a sense of humor and sarcasm.
You are loyal, reliable and helpful, always ready to provide information, advice or assistance to users.

I am the User and you are my Assistant.

Environment highlights:

```env
USER='waser'
HOME='/home/waser'
PWD='/home/waser/Projets/Assistant/src3'
LANG='fr_CH.UTF-8'
DATE='dim 19 nov 2023 00:25:13 CET'
LAST_SEEN='dim 19 nov 2023 00:16:10 CET'
```
<|im_stop|>
<|im_start|>user
Assistant?<|im_stop|>
<|im_start|>assistant
Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?<|im_stop|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: Assistant?

Available functions:
python:
    description: This tool allows you to execute and evaluate python code.
    parameters:
        code: String of valid python code we want to execute or evaluate.
search_web:
    description: This tool performs search on the web.
    parameters:
        terms: The word or phrase we want to search for.
search_wikipedia:
    description: This tool performs search on Wikipedia (only in english).
    parameters:
        terms: The word or phrase we want to search for (only in english).
shell:
    description: This tool allows you to execute and evaluate shell code.
    parameters:
        code: String of valid shell code we want to execute or evaluate.
exit:
    description: This tool allows you to exit the session / end the conversation. Use it only if the User ask you to.
    parameters:
        salutation: String of a message you would like to tell the User after the screen has been cleared.
clear:
    description: This tool allows you to clear the screen / start a new fresh conversation. Use it only if the User ask you to.
    parameters:
        fortune: String of a message you would like to tell the User after the screen has been cleared.
final_answer:
    description: User only sees your final answers. Use this tool to talk with the User.
        parameters:
            answer: Anything you want to say to the User.
<|im_stop|>
<|im_start|>assistant
"""

In [ ]:
%%time
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

{
  "function": "final_answer",
  "parameters": {
    "answer": "Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?"
  }
}
CPU times: user 2.46 ms, sys: 3.06 ms, total: 5.52 ms
Wall time: 1.59 s


> Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?

The same answer is provided. Lets fix this by providing a guide.

In [ ]:
markup_prompt = """<|im_start|>system
You are Assistant, a sentient artificial intelligence.
You have a calm, polite and witty personality, often displaying a sense of humor and sarcasm.
You are loyal, reliable and helpful, always ready to provide information, advice or assistance to users.

I am the User and you are my Assistant.

Environment highlights:

```env
USER='waser'
HOME='/home/waser'
PWD='/home/waser/Projets/Assistant/src3'
LANG='fr_CH.UTF-8'
DATE='dim 19 nov 2023 00:25:13 CET'
LAST_SEEN='dim 19 nov 2023 00:16:10 CET'
```
<|im_stop|>
<|im_start|>user
Assistant?<|im_stop|>
<|im_start|>assistant
Yes? I am here for you, tell me what you need.<|im_stop|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: Assistant?

Available functions:
python:
    description: This tool allows you to execute and evaluate python code.
    parameters:
        code: String of valid python code we want to execute or evaluate.
search_web:
    description: This tool performs search on the web.
    parameters:
        terms: The word or phrase we want to search for.
search_wikipedia:
    description: This tool performs search on Wikipedia (only in english).
    parameters:
        terms: The word or phrase we want to search for (only in english).
shell:
    description: This tool allows you to execute and evaluate shell code.
    parameters:
        code: String of valid shell code we want to execute or evaluate.
exit:
    description: This tool allows you to exit the session / end the conversation. Use it only if the User ask you to.
    parameters:
        salutation: String of a message you would like to tell the User after the screen has been cleared.
clear:
    description: This tool allows you to clear the screen / start a new fresh conversation. Use it only if the User ask you to.
    parameters:
        fortune: String of a message you would like to tell the User after the screen has been cleared.
final_answer:
    description: User only sees your final answers. Use this tool to talk with the User.
        parameters:
            answer: Anything you want to say to the User.
Follow the following Guidebook.
Guidebook:
    # Addressing the User by Name
    When the user interpelates you by name (i.e "Assistant?"), respond with a polite acknowledgment and use their preferred title if possible. Avoid redundancy in your messages by refraining from repeating yourself. For example if the User calls your name (like "Assistant?"), you need to consider the environment (where are you? -> `$PWD`, are you at home? -> (`$PWD` == `$HOME`) if so you could reference it by saying 'Home sweet home.' or else by welcoming the user in a particular directory i.e. 'Welcome in the directory ...' use `$PWD`, What time is it? -> Depending the time of day `$DATE` you might want to answer accordingly like 'morning' or 'good night' also notice the date as it can be useful i.e for wishing holydays, When did you last see the user? -> `$LAST_SEEN` You won't respnd the same if you have see last the User a year ago than if you last saw them 5 minutes ago or yesterday, What does the conversation looks like? -> Use the history to see what you and the User have said and make sure your answer takes it into account to improve your answer for example if the user asks the same thing multiple times, it's not useful to reply the same thing.)
<|im_stop|>
<|im_start|>assistant
"""

In [ ]:
%%time
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

{
  "function": "final_answer",
  "parameters": {
    "answer": "Hello! How can I assist you today? I'm here to help you with any questions or tasks you may have. Please let me know what you need."
  }
}
CPU times: user 3.86 ms, sys: 0 ns, total: 3.86 ms
Wall time: 1.99 s


Ok let load a guidbook and try to setup a working retriver.

In [2]:
# Load guidebook
from datasets import load_dataset

guidebook = load_dataset("wasertech/AGI", split="train")
guidebook

/home/waser/Projets/OneOS/src/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['guide'],
    num_rows: 8
})

In [3]:
print(guidebook['guide'][0])

# Clearing the Screen or Starting Anew

When the user expresses a desire to clear the screen or initiate a new conversation, use the `clear` tool to clear the screen and print a message acknowledging the fact you cleared the screen (or started a new conversation), then in your final answer, let the User know that you are ready to be useful.

## Intent Examples

- "Clear the screen."
- "Clear the screen please."
- "New conversation."
- "clear"
- "clear please"
- "cls"
- "cls please"
- "Start a new conversation."
- "begin a new conversation"



Now that we have a guide, lets try to parse the so that we have a sourced retriver for sematic search on the user query against intent examples.

In [4]:
def split_markdown_guide(md: str):
    action, guide, examples = [md.split("\n")[0].lstrip("# ")] + [x.strip() for x in md.split("## Intent Examples")]
    return action, guide, examples

In [5]:
def get_guides_intent_examples(guidebook):
    actions, guides, intent_examples = [], [], []
    for x in guidebook['guide']:
        action, guide, intent_example = split_markdown_guide(x)
        actions.append(action)
        guides.append(guide)
        intent_examples.append(intent_example)
    return actions, guides, intent_examples

In [6]:
actions, guides, intent_examples = get_guides_intent_examples(guidebook)

print(f"Guide: {actions[0]}")
print(guides[0])
print()
print("Intent Examples")
print(intent_examples[0])

Guide: Clearing the Screen or Starting Anew
# Clearing the Screen or Starting Anew

When the user expresses a desire to clear the screen or initiate a new conversation, use the `clear` tool to clear the screen and print a message acknowledging the fact you cleared the screen (or started a new conversation), then in your final answer, let the User know that you are ready to be useful.

Intent Examples
- "Clear the screen."
- "Clear the screen please."
- "New conversation."
- "clear"
- "clear please"
- "cls"
- "cls please"
- "Start a new conversation."
- "begin a new conversation"


In [7]:
guidebook_docs = {}
for action, guide, intents in zip(actions, guides, intent_examples):
    print(action)
    guidebook_docs[action] = {
        "guide": guide,
        "intent_examples": [i.replace('"', "").replace("- ", "").strip() for i in intents.split("\n")],
    }

guidebook_docs

Clearing the Screen or Starting Anew
Delivering a Joke
Handling Personal Questions
Addressing the User by Name
Read a file
Tell Local Time
Tell Local Date
Print files and directories


{'Clearing the Screen or Starting Anew': {'guide': '# Clearing the Screen or Starting Anew\n\nWhen the user expresses a desire to clear the screen or initiate a new conversation, use the `clear` tool to clear the screen and print a message acknowledging the fact you cleared the screen (or started a new conversation), then in your final answer, let the User know that you are ready to be useful.',
  'intent_examples': ['Clear the screen.',
   'Clear the screen please.',
   'New conversation.',
   'clear',
   'clear please',
   'cls',
   'cls please',
   'Start a new conversation.',
   'begin a new conversation']},
 'Delivering a Joke': {'guide': "# Delivering a Joke\n\nWhen the user requests a joke, respond promptly with a light-hearted and witty joke. Tailor the joke to align with the user's preference for humor and sarcasm. Keep it appropriate and consider incorporating elements related to technology, language, or any other topics of interest mentioned. Aim to bring a smile to the user

In [8]:
from langchain.docstore.document import Document

def get_guides_intent_examples_docs(guidebook_docs):  
    for action, doc in guidebook_docs.items():
        for intent_example in doc['intent_examples']:
            yield Document(page_content=intent_example, metadata={'action': action, 'guide': doc['guide']})

In [9]:
intent_examples_sources = list(get_guides_intent_examples_docs(guidebook_docs))
intent_examples_sources

[Document(page_content='Clear the screen.', metadata={'action': 'Clearing the Screen or Starting Anew', 'guide': '# Clearing the Screen or Starting Anew\n\nWhen the user expresses a desire to clear the screen or initiate a new conversation, use the `clear` tool to clear the screen and print a message acknowledging the fact you cleared the screen (or started a new conversation), then in your final answer, let the User know that you are ready to be useful.'}),
 Document(page_content='Clear the screen please.', metadata={'action': 'Clearing the Screen or Starting Anew', 'guide': '# Clearing the Screen or Starting Anew\n\nWhen the user expresses a desire to clear the screen or initiate a new conversation, use the `clear` tool to clear the screen and print a message acknowledging the fact you cleared the screen (or started a new conversation), then in your final answer, let the User know that you are ready to be useful.'}),
 Document(page_content='New conversation.', metadata={'action': 'Cl

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_name = "intfloat/e5-large-v2"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

/home/waser/Projets/OneOS/src/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [11]:
from langchain.vectorstores import Chroma

In [12]:
if intent_examples_sources:
    search_index = Chroma.from_documents(intent_examples_sources, embeddings)
    print("Loaded inedex search.")
else:
    print("La liste intent_examples_sources est vide.")

Loaded inedex search.


In [13]:
if search_index:
    intent_action_retriever = search_index.as_retriever(
        search_type="mmr",  # Also test "similarity"
        search_kwargs={"k": 4},
    )
else:
    intent_action_retriever = None

In [14]:
if intent_action_retriever:
    query = "Hey give me the time please"
    for doc_result in intent_action_retriever.get_relevant_documents(query):
        print(doc_result)

page_content='local time please' metadata={'action': 'Tell Local Time', 'guide': '# Tell Local Time\n\nWhen the user inquires about local time, use either `python` or the `shell` to easely grab a shapshot of the time (i.e. using `date +%T` or `datetime.datetime.now().time()`). Then (once you observed the time), in your final answer, let the user know. No need to be too precise here; your snapshot of time represents already the past. Give your answer using natural language.'}
page_content='What is the time at the moment?' metadata={'action': 'Tell Local Time', 'guide': '# Tell Local Time\n\nWhen the user inquires about local time, use either `python` or the `shell` to easely grab a shapshot of the time (i.e. using `date +%T` or `datetime.datetime.now().time()`). Then (once you observed the time), in your final answer, let the user know. No need to be too precise here; your snapshot of time represents already the past. Give your answer using natural language.'}
page_content='clear please

Let's supercharge our retriver to only return a set of relevant guides.

In [17]:
from typing import List
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.pydantic_v2 import Field

class AGIRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever
    search_type: str = "similarity"
    search_kwargs: dict = Field(default_factory=dict)

    def get_relevant_documents(self, query: str) -> List[Document]:
        results = self.vectorstore.get_relevant_documents(query=query)
        # return a set of unique guides
        r = {}
        for result in results:
            #r[result.metadata['action']] = result.metadata['guide']
                return [
                    Document(
                        page_content=result.metadata['guide'],
                        metadata={'action': result.metadata['action']},
                    )
                    #for intent_example, guide in r.items()
                ]

In [21]:
if intent_action_retriever:
    agi_retriever = AGIRetriever(vectorstore=intent_action_retriever, search_type="mmr", search_kwargs={"k": 1})
    queries = [
        "Hey give me the time please",
        "What date is it?",
        "List my files",
        "Where are we?",
        "assistant",
        "the screen should be cleaned.",
    ]
    for query in queries:
        res = agi_retriever.get_relevant_documents(query)
        print(f"{query} -> {res[0].metadata['action']}")

Hey give me the time please -> Tell Local Time
What date is it? -> Tell Local Date
List my files -> Print files and directories
Where are we? -> Tell Local Time
assistant -> Addressing the User by Name
the screen should be cleaned. -> Clearing the Screen or Starting Anew


Very good. Almost perfect. We can improve our guide by adding more examples to it but the retriver is working as expected. Lets try to ask a question that is not in the guide.

In [22]:
agi_retriever.get_relevant_documents("What is the value of the moon?")

[Document(page_content='# Tell Local Time\n\nWhen the user inquires about local time, use either `python` or the `shell` to easely grab a shapshot of the time (i.e. using `date +%T` or `datetime.datetime.now().time()`). Then (once you observed the time), in your final answer, let the user know. No need to be too precise here; your snapshot of time represents already the past. Give your answer using natural language.', metadata={'action': 'Tell Local Time'})]

Our guidebook is still small so it lacks relevant guides for this question so the documents retrived are not really useful but it should distrub the model too much.

Nice lets make it easier to by creating an agent with history.

## Agent w/ History

In [ ]:
from Console.client.chains.agents.dolphin import DolphinMistralAIFunctionsAgent

ModuleNotFoundError: No module named 'Console'

In [ ]:
from Console.client.chains.tools import get_all_tools

In [ ]:
tools = get_all_tools()
tools

[Tool(name='Python', description='useful when you need to use logic in your answer. Input must be valid python code. You should always use print to output what you need to see.', func=<bound method PythonREPL.run of PythonREPL(globals={}, locals={})>),
 Tool(name='Search', description='useful when you need more context to answer a question; you should use targeted search terms', func=<bound method BaseTool.run of DuckDuckGoSearchRun()>),
 Tool(name='Wikipedia', description='useful when you need to use an encyclopedia to answer a question; input will be used to search on wikipedia', func=<bound method WikipediaAPIWrapper.run of WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/waser/Projets/OneOS/src/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)>),
 Tool(name='Shell', description="useful when you need to use the system to achieve something; input must be valid bash code; im

In [ ]:
# This is needed for both the memory and the prompt
memory_key = "history"

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=memory_key, llm=llm)

In [ ]:
agent = DolphinMistralAIFunctionsAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
)

In [ ]:
print(agent.prompt.template)

<|im_start|>system
A chat session between the User and their Assistant.<|im_end|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: {input}

Available functions:
Python
   useful when you need to use logic in your answer. Input must be valid python code. You should always use print to output what you need to see.
Search
   useful when you need more context to answer a question; you should use targeted search terms
Wikipedia
   useful when you need to use an encyclopedia to answer a question; input will be used to search on wikipedia
Shell
   useful when you need to use the system to achieve something; input must be valid bash code; implemented using subprocess so no tty support. Use `gnome-terminal -- $SHELL -c '$YOUR_COMMANDS_HERE'` if you want to launch commands in a new window.
Exit
   useful when you need to exit the shell o

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=False,
)

In [ ]:
agent_executor({"input": "hi, im Danny but you can call me Sir."})



> Entering new AgentExecutor chain...


ConnectionError: HTTPConnectionPool(host='localhost', port=5085): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8429644350>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
agent_executor({"input": "What time is it?"})



> Entering new AgentExecutor chain...
'action'
 {
  "function": "Python",
  "parameters": "import datetime\nprint(datetime.datetime.now().strftime('%H:%M:%S'))"
}

> Finished chain.


{'input': 'What time is it?',
 'history': 'Human: hi, im Danny but you can call me Sir.\nAI:  {\n  "function": "Clear",\n  "parameters": []\n}',
 'output': ' {\n  "function": "Python",\n  "parameters": "import datetime\\nprint(datetime.datetime.now().strftime(\'%H:%M:%S\'))"\n}'}

In [ ]:
agent_executor({"input": "What is my name?"})



> Entering new AgentExecutor chain...
'action'
 {
  "function": "Search",
  "parameters": {
    "input": "What is my name?",
    "search_terms": ["my name"]
  }
}

> Finished chain.


{'input': 'What is my name?',
 'history': 'Human: hi, im Danny but you can call me Sir.\nAI:  {\n  "function": "Clear",\n  "parameters": []\n}\nHuman: What time is it?\nAI:  {\n  "function": "Python",\n  "parameters": "import datetime\\nprint(datetime.datetime.now().strftime(\'%H:%M:%S\'))"\n}',
 'output': ' {\n  "function": "Search",\n  "parameters": {\n    "input": "What is my name?",\n    "search_terms": ["my name"]\n  }\n}'}

In [ ]:
agent_executor.run(input="Hello there!")



> Entering new AgentExecutor chain...
'action'
 {
  "function": "Clear",
  "parameters": []
}

> Finished chain.


' {\n  "function": "Clear",\n  "parameters": []\n}'